# Горшочек, вари! или как добиться быстродействия от Julia.

Писать буду по мотивам http://www.stochasticlifestyle.com/7-julia-gotchas-handle/, потому как написано здраво и грамотно. 

Начать, скорее всего, стоит с того, чтобы разобраться, что такое функция в Julia и чем она отличается от метода? В математике функцией называют отображение из одного множества в другое (каждому элементу первого множества ставится в соответствие некоторый элемент второго).

Важно тут то, что функция задается не только правилом, но и своей областью определения (из какого множества и куда действует). То есть, формально говоря, функция сложения на натуральных числах $(x, y) \to x + y$ и функция сложения на действительных числах - это две $\textbf{разные}$ функции! Хотя и обозначаются одним и тем же значком $+$.

С Julia та же история. Функция в Julia - это не в прямом (математическом) смысле функция; скорее, она является именем для целого набора методов, связанных с ней. Методы уже больше соответствуют математическим функциям - в идеале они принимают на вход какие-то аргументы определенного типа (sic!) и возвращают аргументы определенного типа (sic!). Однако при программировании это не всегда так, и ниже мы рассмотрим примеры.

Пока что в доказательство моих слов покажу следующее:

In [3]:
methods(+)

# 180 methods for generic function "+":
+(x::Bool, z::Complex{Bool}) in Base at complex.jl:232
+(x::Bool, y::Bool) in Base at bool.jl:89
+(x::Bool) in Base at bool.jl:86
+(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:96
+(x::Bool, z::Complex) in Base at complex.jl:239
+(a::Float16, b::Float16) in Base at float.jl:372
+(x::Float32, y::Float32) in Base at float.jl:374
+(x::Float64, y::Float64) in Base at float.jl:375
+(z::Complex{Bool}, x::Bool) in Base at complex.jl:233
+(z::Complex{Bool}, x::Real) in Base at complex.jl:247
+(x::Char, y::Integer) in Base at char.jl:40
+(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:312
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:334
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:327
+(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:321
+(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:289
+(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:346
+(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:362
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:460
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:453
+(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:447
+(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:308
+(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:277
+(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:284
+(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:292
+(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:300
+(B::BitArray{2}, J::UniformScaling) in Base.LinAlg at linalg/uniformscaling.jl:59
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem, b::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:87
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuild, b::Base.Pkg.Resolve.VersionWeights.VWPreBuild) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:135
+(a::Base.Pkg.Resolve.VersionWeights.VersionWeight, b::Base.Pkg.Resolve.VersionWeights.VersionWeight) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:197
+(a::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue, b::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue) in Base.Pkg.Resolve.MaxSum.FieldValues at pkg/resolve/fieldvalue.jl:44
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.CompoundPeriod) in Base.Dates at dates/periods.jl:349
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.Period) in Base.Dates at dates/periods.jl:347
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.TimeType) in Base.Dates at dates/periods.jl:387
+(x::Date, y::Base.Dates.Day) in Base.Dates at dates/arithmetic.jl:77
+(x::Date, y::Base.Dates.Week) in Base.Dates at dates/arithmetic.jl:75
+(dt::Date, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:58
+(dt::Date, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:32
+(dt::Date, t::Base.Dates.Time) in Base.Dates at dates/arithmetic.jl:20
+(t::Base.Dates.Time, dt::Date) in Base.Dates at dates/arithmetic.jl:24
+(x::Base.Dates.Time, y::Base.Dates.TimePeriod) in Base.Dates at dates/arithmetic.jl:81
+(dt::DateTime, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:52
+(dt::DateTime, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:28
+(x::DateTime, y::Base.Dates.Period) in Base.Dates at dates/arithmetic.jl:79
+(y::AbstractFloat, x::Bool) in Base at bool.jl:98
+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:32
+(x::Integer, y::Ptr) in Base at pointer.jl:128
+(z::Complex, w::Complex) in Base at complex.jl:221
+(z::Complex, x::Bool) in Base at complex.jl:240
+(x::Real, z::Complex{Bool}) in Base at complex.jl:246
+(x::Real, z::Complex) in Base at complex.jl:258
+(z::Complex, x::Real) in Base at complex.jl:259
+(x::Rational, y::Rational) in Base at rational.jl:245

Под личиной "обычного" плюса скрывается аж 180 методов - сложения булевых, целых чисел, с плавающей точкой, больших, рациональных, матриц различного типа... Не счесть их!

Сделано это для того, чтобы в каждом конкретном случае можно было бы сгенерировать максимально шустрый код. 

In [4]:
# код для целых чисел
@code_native 2 + 3

	.text
Filename: int.jl
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 32
	leaq	(%rdi,%rsi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


In [5]:
# код для чисел с плавающей точкой 
@code_native 2.0 + 3.0

	.text
Filename: float.jl
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 375
	addsd	%xmm1, %xmm0
	popq	%rbp
	retq
	nopw	(%rax,%rax)


Таким образом, Julia, видя аргументы, которые пришли на вход "функции", вызывает самый подходящий для данного случая метод. Всё красиво, пока мы не начинаем сами пытаться что-то изобразить:

In [20]:
y = 1
function plus_value(x)
    return x + y
end

plus_value (generic function with 1 method)

In [21]:
@code_native plus_value(1)

	.text
Filename: In[20]
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r14
	pushq	%rbx
	subq	$48, %rsp
	movq	%fs:0, %rbx
	addq	$-10888, %rbx           # imm = 0xD578
	leaq	-40(%rbp), %r14
	xorps	%xmm0, %xmm0
	movups	%xmm0, -40(%rbp)
	movq	$0, -24(%rbp)
	movq	$6, -56(%rbp)
	movq	(%rbx), %rax
	movq	%rax, -48(%rbp)
	leaq	-56(%rbp), %rax
	movq	%rax, (%rbx)
Source line: 3
	movabsq	$140016279326248, %rax  # imm = 0x7F5814978E28
	movq	(%rax), %rcx
	movq	%rcx, -24(%rbp)
	addq	$339962000, %rax        # imm = 0x14436890
	movq	%rax, -40(%rbp)
	movabsq	$jl_box_int64, %rax
	callq	*%rax
	movq	%rax, -32(%rbp)
	movabsq	$jl_apply_generic, %rax
	movl	$3, %esi
	movq	%r14, %rdi
	callq	*%rax
	movq	-48(%rbp), %rcx
	movq	%rcx, (%rbx)
	addq	$48, %rsp
	popq	%rbx
	popq	%r14
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


Спрашивается: ЧТО ТАК СЛОЖНААА?? Я всего-то и хотел, что 1 прибавить.

Оказывается, не всё так просто. Любая переменная, которую мы заводим в REPLe (интерактивная оболочка Julia), по умолчанию является глобальной. Это значит, что какая угодно другая функция её может изменить, в частности, может изменить её тип, и всё это - прямо во время вычисления функции plus_value (параллельность)! Поэтому Julia приходится "выкручиваться" и оборачивать y в специальные одежды, чтобы её можно было обрабатывать вне зависимости, какого она типа (что-то наподобие PyObject-a в Питоне, с которыми приходится "бороться", когда мы переводим Python в Cython).

Ещё раз: любая переменная, которая заведена в REPLe, является глобальной. 

Как нам исправить данный пример?

In [23]:
# вариант 1 : передаем сам y вместе с x в функцию
y = 1
function plus_value_spec(x, y)
    return x + y
end

plus_value_spec (generic function with 1 method)

In [24]:
@code_native plus_value_spec(1, 1)

	.text
Filename: In[23]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 4
	leaq	(%rdi,%rsi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


In [33]:
# вариант 2 : объявлять y не в REPLe, а внутри самой функции
function plus_value_spec2(x)
    y = 1
    return x + y
end

plus_value_spec2 (generic function with 1 method)

In [34]:
@code_native plus_value_spec2(1)

	.text
Filename: In[33]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 4
	leaq	1(%rdi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


In [36]:
# вариант 3 : "пообещать" компилятору, что переменная у не будет менять ТИП
# объявив её с ключевым словом const
const yConst = 1
function plus_value_spec3(x)
    return x + yConst
end

plus_value_spec3 (generic function with 1 method)

In [37]:
@code_native plus_value_spec3(1)

	.text
Filename: In[36]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 5
	leaq	1(%rdi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


Ещё одна частая проблема, с которой приходится "бороться" (на самом деле, никакой борьбы нет, нужно просто привыкнуть мысли по-Juliaновски) - type instability, нестабильность типов. Возникает, когда в процессе работы функции переменная внутри неё меняет тип:

In [38]:
function count_sum(m)
    s = 0
    for i in 1 : m
        s += 1/i
    end
    return s
end

count_sum (generic function with 1 method)

In [39]:
@code_native count_sum(4)

	.text
Filename: In[38]
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	subq	$136, %rsp
	movq	%rdi, -64(%rbp)
	movq	%fs:0, %rcx
	addq	$-10888, %rcx           # imm = 0xD578
	xorpd	%xmm0, %xmm0
	movupd	%xmm0, -128(%rbp)
	movq	$0, -112(%rbp)
	movupd	%xmm0, -152(%rbp)
	movq	$12, -168(%rbp)
	movq	(%rcx), %rax
	movq	%rax, -160(%rbp)
	leaq	-168(%rbp), %rax
	movq	%rcx, -48(%rbp)
	movq	%rax, (%rcx)
	movq	$0, -136(%rbp)
	movb	$1, %dl
	xorl	%edi, %edi
	movq	%rsi, -104(%rbp)
Source line: 3
	testq	%rsi, %rsi
	jle	L464
	movb	$1, %dl
	movabsq	$140016619271792, %rax  # imm = 0x7F5828DAB670
	xorl	%r12d, %r12d
Source line: 4
	leaq	-334868576(%rax), %rcx
	movq	%rcx, -96(%rbp)
	leaq	43053984(%rax), %rcx
	movq	%rcx, -88(%rbp)
	leaq	16456(%rax), %rax
	movq	%rax, -80(%rbp)
	movabsq	$140016354305096, %rax  # imm = 0x7F58190FA448
	movsd	(%rax), %xmm0           # xmm0 = mem[0],zero
	movsd	%xmm0, -72(%rbp)
	xorl	%edi, %edi
	nopw	%cs:(%rax,%rax)
L208:
	cmpb	$1, %dl
	movq

Если вы видите, что ваша крошечная функция в результате компиляции превратилась в такого левиафана - знайте, это не правы ВЫ, а не Julia! В чём же проблема?

In [40]:
@code_warntype count_sum(4)

Variables:
  #self#::#count_sum
  m::Int64
  i::Int64
  #temp#@_4::Int64
  s::Union{Float64, Int64}
  #temp#@_6::Core.MethodInstance
  #temp#@_7::Float64

Body:
  begin 
      s::Union{Float64, Int64} = 0 # line 3:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1, m::Int64)::Bool, m::Int64, (Base.sub_int)(1, 1)::Int64)::Int64
      #temp#@_4::Int64 = 1
      5: 
      unless (Base.not_int)((#temp#@_4::Int64 === (Base.add_int)(SSAValue(2), 1)::Int64)::Bool)::Bool goto 30
      SSAValue(3) = #temp#@_4::Int64
      SSAValue(4) = (Base.add_int)(#temp#@_4::Int64, 1)::Int64
      i::Int64 = SSAValue(3)
      #temp#@_4::Int64 = SSAValue(4) # line 4:
      unless (s::Union{Float64, Int64} isa Int64)::Bool goto 15
      #temp#@_6::Core.MethodInstance = MethodInstance for +(::Int64, ::Float64)
      goto 24
      15: 
      unless (s::Union{Float64, Int64} isa Float64)::Bool goto 19
      #temp#@_6::Core.MethodInstance = MethodInstance for +(::Float64, ::Float64)
      goto 24
      19: 

code_warntype - палочка-выручалочка для тех случаев, когда "должно работать быстро, а работает медленно, ПОЧЕМУУУ???". Проблемные места сразу же подсвечиваются. Как видите, ответ в том, что вначале мы заводим $s$ как переменную типа Int64, а потом начинаем к ней прибавлять Float-ы, и Julia не понимает, какое значение в итоге надо вернуть - Float64 или же Int64? Немного подумав, мы пишем:

In [41]:
function count_sum_spec(m)
    s = 0.0
    for i in 1 : m
        s += 1/i
    end
    return s
end

count_sum_spec (generic function with 1 method)

In [43]:
@code_warntype count_sum_spec(4)

Variables:
  #self#::#count_sum_spec
  m::Int64
  i::Int64
  #temp#::Int64
  s::Float64

Body:
  begin 
      s::Float64 = 0.0 # line 3:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1, m::Int64)::Bool, m::Int64, (Base.sub_int)(1, 1)::Int64)::Int64
      #temp#::Int64 = 1
      5: 
      unless (Base.not_int)((#temp#::Int64 === (Base.add_int)(SSAValue(2), 1)::Int64)::Bool)::Bool goto 15
      SSAValue(3) = #temp#::Int64
      SSAValue(4) = (Base.add_int)(#temp#::Int64, 1)::Int64
      i::Int64 = SSAValue(3)
      #temp#::Int64 = SSAValue(4) # line 4:
      s::Float64 = (Base.add_float)(s::Float64, (Base.div_float)((Base.sitofp)(Float64, 1)::Float64, (Base.sitofp)(Float64, i::Int64)::Float64)::Float64)::Float64
      13: 
      goto 5
      15:  # line 6:
      return s::Float64
  end::Float64


Совсем другое дело! Мы сразу завели переменную s как тип Float, и всё "просто заработало"! Важный вывод состоит вот в чем: если мы хотим, чтобы всё работало быстро - нужно следить за стабильностью типов (чтобы функция возвращала один конкретный тип, чтобы в code_warntype не было типов Union и Any). 

### View vs Copy
Теперь попробуем выполнить такую нехитрую задачу - сосчитать сумму элементов массива на четных местах.

In [93]:
# заводим массив
arr = rand(100000);

In [94]:
# попробуем для начала с помощью slice-ов
@benchmark res = sum(arr[2 : 2 : 100000])

BenchmarkTools.Trial: 
  memory estimate:  390.81 KiB
  allocs estimate:  6
  --------------
  minimum time:     101.070 μs (0.00% GC)
  median time:      112.106 μs (0.00% GC)
  mean time:        132.399 μs (8.42% GC)
  maximum time:     1.342 ms (67.32% GC)
  --------------
  samples:          10000
  evals/sample:     1

Спрашивается - откуда взялись эти 390 Килобайт выделения памяти? Пробуем переписать всё вручную:

In [95]:
function sum_even(arr)
    s = 0.0
    for i in 2 : 2 : 100000
        s += arr[i]
    end
    return s
end

sum_even (generic function with 1 method)

In [98]:
@benchmark sum_even(arr)

BenchmarkTools.Trial: 
  memory estimate:  16 bytes
  allocs estimate:  1
  --------------
  minimum time:     95.627 μs (0.00% GC)
  median time:      95.856 μs (0.00% GC)
  mean time:        96.274 μs (0.00% GC)
  maximum time:     151.075 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

Совсем другое дело - всего 16 байт!
Секрет в том, что операция ``` arr[i:j] ``` создает копию массива и над ней проделывает операции. Чтобы не делать копий массива, нужно использовать функцию view

In [101]:
@benchmark arr[2 : 2 : 100000]

BenchmarkTools.Trial: 
  memory estimate:  390.80 KiB
  allocs estimate:  5
  --------------
  minimum time:     72.382 μs (0.00% GC)
  median time:      79.656 μs (0.00% GC)
  mean time:        99.800 μs (12.31% GC)
  maximum time:     1.152 ms (81.14% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [102]:
@benchmark view(arr, 2 : 2 : 100000)

BenchmarkTools.Trial: 
  memory estimate:  544 bytes
  allocs estimate:  17
  --------------
  minimum time:     2.053 μs (0.00% GC)
  median time:      2.131 μs (0.00% GC)
  mean time:        2.264 μs (2.59% GC)
  maximum time:     310.925 μs (96.53% GC)
  --------------
  samples:          10000
  evals/sample:     10

Таким образом, с помощью view мы можем переписать нашу функцию следующим образом:

In [103]:
res = sum(view(arr, 2 : 2 : 100000))

25081.71381638191

In [104]:
@benchmark res = sum(view(arr, 2 : 2 : 100000))

BenchmarkTools.Trial: 
  memory estimate:  560 bytes
  allocs estimate:  18
  --------------
  minimum time:     95.479 μs (0.00% GC)
  median time:      95.743 μs (0.00% GC)
  mean time:        96.523 μs (0.00% GC)
  maximum time:     149.117 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

То же самое может быть сделано и с помощью макроса views

In [105]:
@views res = sum(arr[2 : 2 : 100000])

25081.71381638191

In [106]:
@benchmark @views res = sum(arr[2 : 2 : 100000])

BenchmarkTools.Trial: 
  memory estimate:  640 bytes
  allocs estimate:  21
  --------------
  minimum time:     95.877 μs (0.00% GC)
  median time:      96.245 μs (0.00% GC)
  mean time:        97.315 μs (0.00% GC)
  maximum time:     157.612 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

И ещё одна хитрость. Предположим, что мы хотим записать в массив arr сумму двух других массивов:

In [113]:
arr2 = rand(100_000);
arr3 = rand(100_000);

In [114]:
@benchmark arr = arr2 + arr3

BenchmarkTools.Trial: 
  memory estimate:  781.33 KiB
  allocs estimate:  2
  --------------
  minimum time:     223.237 μs (0.00% GC)
  median time:      403.268 μs (0.00% GC)
  mean time:        448.208 μs (8.54% GC)
  maximum time:     4.265 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

Как всегда, большое выделение памяти сигнализирует нам, что "что-то пошло не так". Оказывается, тут вот что происходит: Julia выделяет новый массив под сумму arr2 + arr3, и потом делает так, что имя arr ссылается на данный новый массив. Но у нас уже есть массив arr! Мы просто хотим записать туда сумму двух других, выделять ничего не нужно. Для этого нужно сделать присваивание немного другим:

In [115]:
# всё ещё не работает - знак + без точки
@benchmark arr .= arr2 + arr3

BenchmarkTools.Trial: 
  memory estimate:  781.33 KiB
  allocs estimate:  2
  --------------
  minimum time:     351.445 μs (0.00% GC)
  median time:      405.853 μs (0.00% GC)
  mean time:        447.267 μs (6.10% GC)
  maximum time:     1.790 ms (58.44% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [116]:
# заработало!
@benchmark arr .= arr2 .+ arr3

BenchmarkTools.Trial: 
  memory estimate:  96 bytes
  allocs estimate:  4
  --------------
  minimum time:     116.111 μs (0.00% GC)
  median time:      132.235 μs (0.00% GC)
  mean time:        145.294 μs (0.00% GC)
  maximum time:     533.085 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

Это не все "секреты" языка, позволяющие улучшить производительность, но бОльшая часть проблем возникает именно из-за первых двух причин - нестабильность типов и глобальные переменные. За этим легко следить, используя макросы @code_warntype (указывает на нестабильность типов и в т.ч. глобальные переменные), @benchmark (показывает усредненное время выполнения функции и количество выделяемой памяти; большие объемы выделяемой памяти свидетельствуют о том, что, скорее всего, что-то идёт не так).

Непокрытыми (пока что) остались такие темы, как:
1. Проброс предвыделенного (preallocated) массива в функцию. Если функция часто вызывается (особенно во внутреннем частом цикле) и аллокирует массивы, то это может служить причиной замедления времени (выделить память, очистить в конце - всё это небыстро). Иногда выгоднее "пробрасывать" как один из аргументов массив tmp, в котором будут храниться промежуточные вычисления, выделяя его вне внутреннего цикла.
2. Макрос @inbounds. Иногда, если мы знаем, что цикл идет точно по границе массива и за его границы мы никак не выйдем, то можно ставить макрос в виде ```@inbounds for i in 1 : k```, что говорит компилятору, что не нужно на каждом шаге проверять, не произошел ли выход за границы массива. Это может очень существенно ускорять процесс обработки массивов, особенно если использовать во внутренних (вложенных) циклах.
3. Обход двумерных массивов. Julia хранит двумерные массивы по столбцам, а не по строкам. Следовательно, и обрабатывать их быстрее будет, если ходить сначала по столбцу (внутренний цикл), а затем по строке. То есть
```julia 
for j in 1 : numCols
    for i in 1 : numRows
        do_something(a[i, j])
    end
end
```